In [27]:
import pandas as pd
import numpy as np
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

def create_new_visit_area_mapping(df_visit, df_photo=None, custom_residence_keywords=None, enable_residence_grouping=True):
    """
    방문지 정보에서 장소명만을 기반으로 새로운 ID 매핑을 생성
    '집'이 포함된 장소는 모두 하나로 통합 (횟집 등은 제외)
    
    Args:
        df_visit: 방문지 정보 DataFrame
        df_photo: 관광사진 DataFrame (선택사항, 추가 ID 포함 가능)
        custom_residence_keywords: 사용자 정의 거주지 키워드 리스트
        enable_residence_grouping: 거주지 통합 기능 사용 여부
    
    Returns:
        tuple: (원본 ID -> 새 ID 매핑 딕셔너리, 위치 정보 맵, 매핑 상세 정보)
    """
    print("=== NEW_VISIT_AREA_ID 매핑 생성 시작 ===")
    print("※ 장소명만으로 매핑 (주소 미사용)")
    
    # 1. 데이터 복사 및 전처리
    df = df_visit.copy()
    
    # 관광사진 데이터에만 있는 추가 ID 처리
    if df_photo is not None:
        photo_only_ids = set(df_photo['VISIT_AREA_ID'].unique()) - set(df['VISIT_AREA_ID'].unique())
        if photo_only_ids:
            print(f"\n관광사진에만 있는 ID: {len(photo_only_ids)}개")
            
            # 관광사진의 고유 ID와 장소명 추출
            photo_unique = df_photo[df_photo['VISIT_AREA_ID'].isin(photo_only_ids)][['VISIT_AREA_ID', 'VISIT_AREA_NM']].drop_duplicates()
            
            # 방문지 데이터에 추가
            for _, row in photo_unique.iterrows():
                if pd.notna(row['VISIT_AREA_NM']):
                    new_row = pd.Series({
                        'VISIT_AREA_ID': row['VISIT_AREA_ID'],
                        'VISIT_AREA_NM': row['VISIT_AREA_NM'],
                        'ROAD_NM_ADDR': '',
                        'LOTNO_ADDR': '',
                        'X_COORD': None,
                        'Y_COORD': None
                    })
                    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    
    # 거주지 관련 키워드 정의 - '집'만 사용
    if custom_residence_keywords:
        residence_keywords = custom_residence_keywords
    else:
        residence_keywords = ['집']
    
    def is_residence(name):
        """장소명이 거주지인지 확인 - '집'만 체크 (횟집 등 제외)"""
        if not name or not enable_residence_grouping:
            return False
        
        # 횟집, 술집 등은 제외
        exclude_patterns = ['횟집', '술집', '찜질', '커피', '볶는', '게스트']
        for pattern in exclude_patterns:
            if pattern in name:
                return False
        
        # '집'이 단독으로 쓰이거나 특정 패턴에 해당하는 경우만
        residence_patterns = [
            name == '집',                    # 정확히 '집'
            name.endswith(' 집'),            # '우리 집', '친구 집' 등
            name.endswith('집)'),            # '부모님 집)' 등
            '부모' in name and '집' in name, # '부모님 집'
            '친척' in name and '집' in name, # '친척 집'
            '우리집' in name,                # '우리집'
            '본가' in name,                  # '본가'
            name.startswith('집 '),          # '집 앞' 등
        ]
        
        return any(residence_patterns)
    
    if enable_residence_grouping:
        print(f"거주지 키워드: {residence_keywords}")
    else:
        print("거주지 통합 기능이 비활성화되었습니다.")
    
    # 2. 각 ID별 정보 수집
    id_info = {}
    
    for _, row in df.iterrows():
        visit_area_id = row['VISIT_AREA_ID']
        name = str(row['VISIT_AREA_NM']).strip() if pd.notna(row['VISIT_AREA_NM']) else ''
        road_addr = str(row['ROAD_NM_ADDR']).strip() if pd.notna(row['ROAD_NM_ADDR']) else ''
        lot_addr = str(row['LOTNO_ADDR']).strip() if pd.notna(row['LOTNO_ADDR']) else ''
        
        # 주소 정보는 참고용으로만 저장
        if road_addr.lower() == 'nan':
            road_addr = ''
        if lot_addr.lower() == 'nan':
            lot_addr = ''
        
        id_info[visit_area_id] = {
            'name': name,
            'road_addr': road_addr,  # 참고용
            'lot_addr': lot_addr,    # 참고용
            'x_coord': row.get('X_COORD'),
            'y_coord': row.get('Y_COORD'),
            'is_residence': is_residence(name)
        }
    
    # 3. 고유 장소 식별 및 매핑 생성
    location_map = {}
    id_to_location_key = {}
    mapping_details = []  # 매핑 상세 정보 저장
    
    # 3-1. 거주지 처리
    residence_ids = []
    if enable_residence_grouping:
        for visit_area_id, info in id_info.items():
            if info['is_residence']:
                residence_ids.append(visit_area_id)
                id_to_location_key[visit_area_id] = "RESIDENCE_INTEGRATED"
                
                mapping_details.append({
                    'VISIT_AREA_ID': visit_area_id,
                    'VISIT_AREA_NM': info['name'],
                    'ROAD_NM_ADDR': info['road_addr'],
                    'LOTNO_ADDR': info['lot_addr'],
                    'LOCATION_KEY': "RESIDENCE_INTEGRATED",
                    'IS_RESIDENCE': True
                })
        
        if residence_ids:
            location_map["RESIDENCE_INTEGRATED"] = {
                'name': '거주지(집_통합)',
                'original_ids': residence_ids,
                'x_coord': None,
                'y_coord': None
            }
            print(f"거주지 통합 완료: {len(residence_ids)}개 ID를 하나로 통합")
            
            # 통합된 거주지 이름들 출력
            residence_names = []
            residence_name_counts = {}
            for rid in residence_ids:
                if rid in id_info:
                    name = id_info[rid]['name']
                    residence_names.append(name)
                    residence_name_counts[name] = residence_name_counts.get(name, 0) + 1
            
            print(f"\n통합된 거주지 종류:")
            for name, count in sorted(residence_name_counts.items(), key=lambda x: x[1], reverse=True)[:10]:
                print(f"  - {name}: {count}개")
    
    # 3-2. 일반 장소 처리 (거주지가 아닌 것들)
    for visit_area_id, info in id_info.items():
        # 거주지는 이미 처리됨
        if enable_residence_grouping and info['is_residence']:
            continue
        
        name = info['name']
        
        # 이름이 없는 경우 처리
        if not name:
            name = f"이름없음_ID{visit_area_id}"
        
        # 고유 키 생성 (장소명만 사용)
        location_key = name
        
        # 위치 정보 저장
        if location_key not in location_map:
            location_map[location_key] = {
                'name': name,
                'original_ids': [],
                'x_coord': info['x_coord'],
                'y_coord': info['y_coord']
            }
        
        location_map[location_key]['original_ids'].append(visit_area_id)
        id_to_location_key[visit_area_id] = location_key
        
        mapping_details.append({
            'VISIT_AREA_ID': visit_area_id,
            'VISIT_AREA_NM': name,
            'ROAD_NM_ADDR': info['road_addr'],
            'LOTNO_ADDR': info['lot_addr'],
            'LOCATION_KEY': location_key,
            'IS_RESIDENCE': False
        })
    
    # 4. NEW_VISIT_AREA_ID 할당 (1부터 시작)
    id_mapping = {}
    location_to_new_id = {}
    new_id_details = []  # 새 ID별 정보 저장
    
    # 정렬하여 일관된 순서 보장
    sorted_locations = sorted(location_map.items())
    
    new_id = 1
    for location_key, info in sorted_locations:
        location_to_new_id[location_key] = new_id
        
        # 해당 위치의 모든 원본 ID를 새 ID로 매핑
        for original_id in info['original_ids']:
            id_mapping[original_id] = new_id
        
        # 새 ID 정보 저장
        new_id_details.append({
            'NEW_VISIT_AREA_ID': new_id,
            'LOCATION_KEY': location_key,
            'VISIT_AREA_NM': info['name'],
            'ORIGINAL_ID_COUNT': len(info['original_ids']),
            'ORIGINAL_IDS': ','.join(map(str, info['original_ids'])),
            'X_COORD': info['x_coord'],
            'Y_COORD': info['y_coord']
        })
        
        new_id += 1
    
    print(f"총 {len(id_mapping)}개의 원본 ID를 {new_id-1}개의 새 ID로 매핑")
    
    # 5. 매핑 상세 정보에 NEW_VISIT_AREA_ID 추가
    for detail in mapping_details:
        detail['NEW_VISIT_AREA_ID'] = id_mapping.get(detail['VISIT_AREA_ID'])

    # 5-1. TRAVEL_ID 추가
    mapping_details = add_travel_id_to_mapping_details(df_visit, mapping_details)

    
    # 6. 중복 통계 출력
    duplicate_locations = [(k, v) for k, v in location_map.items() if len(v['original_ids']) > 1]
    duplicate_locations.sort(key=lambda x: len(x[1]['original_ids']), reverse=True)
    
    print(f"\n동일한 이름으로 통합된 장소: {len(duplicate_locations)}개")
    print("통합이 많이 된 상위 10개 장소:")
    for location_key, info in duplicate_locations[:10]:
        print(f"- {info['name']}: {len(info['original_ids'])}개 ID 통합")
    
    return id_mapping, location_map, {'mapping_details': mapping_details, 'new_id_details': new_id_details}

def apply_new_id_to_visit_data(df_visit, id_mapping):
    """방문지 데이터에 NEW_VISIT_AREA_ID 적용"""
    print("\n=== 방문지 데이터에 NEW_VISIT_AREA_ID 적용 ===")
    
    df_result = df_visit.copy()
    df_result['NEW_VISIT_AREA_ID'] = df_result['VISIT_AREA_ID'].map(id_mapping)
    
    # 매핑되지 않은 ID 확인
    unmapped = df_result[df_result['NEW_VISIT_AREA_ID'].isna()]
    if len(unmapped) > 0:
        print(f"경고: {len(unmapped)}개 행이 매핑되지 않음")
        print("매핑되지 않은 VISIT_AREA_ID:", unmapped['VISIT_AREA_ID'].unique()[:10])
    
    print(f"매핑 완료: {len(df_result)}개 행")
    return df_result

def apply_new_id_to_move_data(df_move, id_mapping):
    """이동내역 데이터에 NEW_VISIT_AREA_ID 적용"""
    print("\n=== 이동내역 데이터에 NEW_VISIT_AREA_ID 적용 ===")
    
    df_result = df_move.copy()
    
    # START_VISIT_AREA_ID 매핑
    df_result['NEW_START_VISIT_AREA_ID'] = df_result['START_VISIT_AREA_ID'].map(id_mapping)
    
    # END_VISIT_AREA_ID 매핑  
    df_result['NEW_END_VISIT_AREA_ID'] = df_result['END_VISIT_AREA_ID'].map(id_mapping)
    
    # 매핑 결과 확인
    start_unmapped = df_result[pd.notna(df_result['START_VISIT_AREA_ID']) & 
                              pd.isna(df_result['NEW_START_VISIT_AREA_ID'])]
    end_unmapped = df_result[pd.notna(df_result['END_VISIT_AREA_ID']) & 
                            pd.isna(df_result['NEW_END_VISIT_AREA_ID'])]
    
    if len(start_unmapped) > 0:
        print(f"경고: {len(start_unmapped)}개 START_VISIT_AREA_ID가 매핑되지 않음")
    
    if len(end_unmapped) > 0:
        print(f"경고: {len(end_unmapped)}개 END_VISIT_AREA_ID가 매핑되지 않음")
    
    print(f"매핑 완료: {len(df_result)}개 행")
    
    # 이동 패턴 통계
    print("\n=== 이동 패턴 분석 ===")
    start_only = df_result[pd.notna(df_result['NEW_START_VISIT_AREA_ID']) & 
                          pd.isna(df_result['NEW_END_VISIT_AREA_ID'])]
    end_only = df_result[pd.isna(df_result['NEW_START_VISIT_AREA_ID']) & 
                        pd.notna(df_result['NEW_END_VISIT_AREA_ID'])]
    both = df_result[pd.notna(df_result['NEW_START_VISIT_AREA_ID']) & 
                    pd.notna(df_result['NEW_END_VISIT_AREA_ID'])]
    
    print(f"- 출발지만 있는 경우 (여행 시작): {len(start_only)}개")
    print(f"- 도착지만 있는 경우 (경유지): {len(end_only)}개") 
    print(f"- 출발지+도착지 모두 있는 경우: {len(both)}개")
    
    return df_result

def apply_new_id_to_photo_data(df_photo, id_mapping, df_visit_original):
    """
    관광사진 데이터에 NEW_VISIT_AREA_ID 적용
    VISIT_AREA_NM이 NaN인 경우 방문지 데이터에서 보완
    """
    print("\n=== 관광사진 데이터에 NEW_VISIT_AREA_ID 적용 ===")
    
    df_result = df_photo.copy()
    
    # 1. VISIT_AREA_NM이 NaN인 경우 방문지 데이터에서 보완
    print("장소명 보완 작업 시작...")
    name_mapping = {}
    for _, row in df_visit_original.iterrows():
        visit_id = row['VISIT_AREA_ID']
        visit_name = row['VISIT_AREA_NM']
        if pd.notna(visit_name) and visit_id not in name_mapping:
            name_mapping[visit_id] = visit_name
    
    # NaN인 장소명 보완
    nan_count = df_result['VISIT_AREA_NM'].isna().sum()
    print(f"장소명이 없는 사진: {nan_count}개")
    
    for idx, row in df_result.iterrows():
        if pd.isna(row['VISIT_AREA_NM']) and row['VISIT_AREA_ID'] in name_mapping:
            df_result.at[idx, 'VISIT_AREA_NM'] = name_mapping[row['VISIT_AREA_ID']]
    
    filled_count = nan_count - df_result['VISIT_AREA_NM'].isna().sum()
    print(f"보완된 장소명: {filled_count}개")
    
    # 2. NEW_VISIT_AREA_ID 매핑
    df_result['NEW_VISIT_AREA_ID'] = df_result['VISIT_AREA_ID'].map(id_mapping)
    
    # 매핑되지 않은 ID 확인
    unmapped = df_result[df_result['NEW_VISIT_AREA_ID'].isna()]
    if len(unmapped) > 0:
        print(f"경고: {len(unmapped)}개 행이 매핑되지 않음")
        print("매핑되지 않은 VISIT_AREA_ID 예시:", unmapped['VISIT_AREA_ID'].unique()[:10])
        
        # 매핑되지 않은 ID들의 장소명 확인
        unmapped_with_names = unmapped[unmapped['VISIT_AREA_NM'].notna()]
        if len(unmapped_with_names) > 0:
            print("\n매핑되지 않은 장소명 예시:")
            for _, row in unmapped_with_names.head(5).iterrows():
                print(f"- {row['VISIT_AREA_NM']} (ID: {row['VISIT_AREA_ID']})")
    
    # 통계 출력
    print(f"\n매핑 완료: {len(df_result)}개 사진")
    print(f"총 고유 장소 수 (원본): {df_result['VISIT_AREA_ID'].nunique()}")
    print(f"총 고유 장소 수 (새ID): {df_result['NEW_VISIT_AREA_ID'].nunique()}")
    
    # 장소별 사진 수 확인 (장소명 포함)
    photo_stats = df_result.groupby(['NEW_VISIT_AREA_ID', 'VISIT_AREA_NM'])['TOUR_PHOTO_SEQ'].count().reset_index(name='photo_count')
    photo_stats = photo_stats.sort_values('photo_count', ascending=False)
    
    print(f"\n사진이 많은 상위 5개 장소:")
    for _, row in photo_stats.head().iterrows():
        place_name = row['VISIT_AREA_NM'] if pd.notna(row['VISIT_AREA_NM']) else '이름없음'
        print(f"- {place_name} (NEW_ID: {int(row['NEW_VISIT_AREA_ID'])}): {row['photo_count']}개 사진")
    
    # 거주지 관련 통계
    residence_photos = df_result[df_result['VISIT_AREA_NM'].str.contains('집', na=False)]
    if len(residence_photos) > 0:
        print(f"\n'집' 관련 장소 사진 분포:")
        residence_stats = residence_photos.groupby('NEW_VISIT_AREA_ID')['TOUR_PHOTO_SEQ'].count().sort_values(ascending=False)
        for new_id, count in residence_stats.head().items():
            print(f"- NEW_ID {int(new_id)}: {count}개 사진")
    
    return df_result

def analyze_travel_patterns(df_move_updated):
    """여행 패턴 분석"""
    print("\n=== 여행 패턴 상세 분석 ===")
    
    travel_stats = []
    
    for travel_id in df_move_updated['TRAVEL_ID'].unique():
        travel_data = df_move_updated[df_move_updated['TRAVEL_ID'] == travel_id].sort_values('START_DT_MIN')
        
        start_points = travel_data[pd.notna(travel_data['NEW_START_VISIT_AREA_ID'])]
        end_points = travel_data[pd.notna(travel_data['NEW_END_VISIT_AREA_ID'])]
        
        travel_stats.append({
            'TRAVEL_ID': travel_id,
            'total_moves': len(travel_data),
            'start_points': len(start_points),
            'end_points': len(end_points),
            'unique_locations': len(set(
                list(travel_data['NEW_START_VISIT_AREA_ID'].dropna()) + 
                list(travel_data['NEW_END_VISIT_AREA_ID'].dropna())
            ))
        })
    
    df_travel_stats = pd.DataFrame(travel_stats)
    
    print("여행별 통계:")
    print(f"- 평균 이동 횟수: {df_travel_stats['total_moves'].mean():.1f}")
    print(f"- 평균 방문 장소 수: {df_travel_stats['unique_locations'].mean():.1f}")
    print(f"- 최대 이동 횟수: {df_travel_stats['total_moves'].max()}")
    print(f"- 최대 방문 장소 수: {df_travel_stats['unique_locations'].max()}")
    
    return df_travel_stats

def main(custom_residence_keywords=None, enable_residence_grouping=True):
    """
    메인 실행 함수
    
    Args:
        custom_residence_keywords: 사용자 정의 거주지 키워드 리스트
        enable_residence_grouping: 거주지 통합 기능 사용 여부
    """
    try:
        # 1. 데이터 로드
        print("=== 데이터 로드 시작 ===")
        
        df_visit = pd.read_csv('./merged_csv/방문지_total.csv')
        df_move = pd.read_csv('./merged_csv/fin/이동내역_fin.csv')
        df_photo = pd.read_csv('./merged_csv/관광사진_total.csv')
        
        print(f"방문지 정보: {len(df_visit)} 행, {len(df_visit.columns)} 열")
        print(f"이동내역: {len(df_move)} 행, {len(df_move.columns)} 열")
        print(f"관광사진: {len(df_photo)} 행, {len(df_photo.columns)} 열")
        
        # 2. 데이터 미리보기
        print("\n=== 방문지 정보 샘플 ===")
        print(df_visit[['VISIT_AREA_ID', 'VISIT_AREA_NM', 'ROAD_NM_ADDR', 'LOTNO_ADDR']].head())
        
        print("\n=== 이동내역 샘플 ===")
        print(df_move[['TRAVEL_ID', 'START_VISIT_AREA_ID', 'END_VISIT_AREA_ID']].head())
        
        print("\n=== 관광사진 샘플 ===")
        print(df_photo[['TRAVEL_ID', 'VISIT_AREA_ID', 'VISIT_AREA_NM', 'PHOTO_FILE_NM']].head())
        
        # 3. NEW_VISIT_AREA_ID 매핑 생성 (관광사진 데이터도 포함)
        id_mapping, location_map, extra_info = create_new_visit_area_mapping(
            df_visit, df_photo, custom_residence_keywords, enable_residence_grouping
        )
        
        # 4. 각 데이터에 새 ID 적용
        df_visit_updated = apply_new_id_to_visit_data(df_visit, id_mapping)
        df_move_updated = apply_new_id_to_move_data(df_move, id_mapping)
        df_photo_updated = apply_new_id_to_photo_data(df_photo, id_mapping, df_visit)
        
        # 5. 여행 패턴 분석
        df_travel_stats = analyze_travel_patterns(df_move_updated)
        
        # 6. 결과 저장
        print("\n=== 결과 저장 ===")
        
        # 기존 파일들 저장
        df_visit_updated.to_csv('./merged_csv/fin/방문지_fin.csv', index=False, encoding='utf-8-sig')
        df_move_updated.to_csv('./merged_csv/fin/이동내역_fin.csv', index=False, encoding='utf-8-sig')
        df_photo_updated.to_csv('./merged_csv/fin/관광사진_fin.csv', index=False, encoding='utf-8-sig')
        df_travel_stats.to_csv('./merged_csv/fin/방문지_meta.csv', index=False, encoding='utf-8-sig')
        
        # 새로운 매핑 정보 파일들 저장
        # 1) 매핑 상세 정보 (원본 ID별 매핑 정보)
        df_mapping_details = pd.DataFrame(extra_info['mapping_details'])
        df_mapping_details = df_mapping_details.sort_values(['NEW_VISIT_AREA_ID', 'VISIT_AREA_ID'])
        df_mapping_details.to_csv('./merged_csv/fin/id_mapping_details.csv', index=False, encoding='utf-8-sig')
        
        # 2) 새 ID별 정보 (통합된 장소 정보)
        df_new_id_info = pd.DataFrame(extra_info['new_id_details'])
        df_new_id_info.to_csv('./merged_csv/fin/new_id_summary.csv', index=False, encoding='utf-8-sig')
        
        print("저장 완료:")
        print("- 방문지_fin.csv (방문지 정보 + NEW_VISIT_AREA_ID)")
        print("- 이동내역_fin.csv (이동내역 + NEW_START/END_VISIT_AREA_ID)")
        print("- 관광사진_fin.csv (관광사진 + NEW_VISIT_AREA_ID)")
        print("- 방문지_meta.csv (여행별 통계)")
        print("- id_mapping_details.csv (원본 ID별 매핑 상세 정보)")
        print("- new_id_summary.csv (새 ID별 통합 장소 정보)")
        
        # 7. 최종 요약
        print("\n=== 최종 요약 ===")
        print(f"총 고유 장소 수: {len(location_map)}")
        print(f"NEW_VISIT_AREA_ID 범위: 1 ~ {max(id_mapping.values())}")
        print(f"중복 제거된 장소 수: {len(df_visit['VISIT_AREA_ID'].unique()) - len(location_map)}")
        
        # 매핑 검증
        print("\n=== 매핑 검증 ===")
        print(f"원본 고유 ID 수: {len(df_visit['VISIT_AREA_ID'].unique())}")
        print(f"매핑된 ID 수: {len(id_mapping)}")
        print(f"새로운 고유 ID 수: {len(set(id_mapping.values()))}")
        
        # 관광사진 매핑 요약
        print("\n=== 관광사진 매핑 요약 ===")
        print(f"총 사진 수: {len(df_photo)}")
        print(f"매핑된 사진 수: {df_photo_updated['NEW_VISIT_AREA_ID'].notna().sum()}")
        print(f"고유 장소 수 (원본): {df_photo['VISIT_AREA_ID'].nunique()}")
        print(f"고유 장소 수 (새ID): {df_photo_updated['NEW_VISIT_AREA_ID'].nunique()}")
        
        return {
            'visit_data': df_visit_updated,
            'move_data': df_move_updated,
            'photo_data': df_photo_updated,
            'travel_stats': df_travel_stats,
            'id_mapping': id_mapping,
            'location_map': location_map,
            'mapping_details': df_mapping_details,
            'new_id_info': df_new_id_info
        }
        
    except Exception as e:
        print(f"오류 발생: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# 사용 예시들
def run_with_default_settings():
    """기본 설정으로 실행 (거주지 통합 포함)"""
    return main()

def run_without_residence_grouping():
    """거주지 통합 없이 실행"""
    return main(enable_residence_grouping=False)

def run_with_custom_keywords():
    """사용자 정의 키워드로 실행"""
    custom_keywords = ['집', '숙소']  # 집과 숙소만 거주지로 처리
    return main(custom_residence_keywords=custom_keywords)

def check_residence_places(df_visit):
    """'집'이 포함된 장소들을 미리 확인하는 함수"""
    print("\n=== '집'이 포함된 장소 확인 ===")
    
    residence_locations = []
    for _, row in df_visit.iterrows():
        name = str(row['VISIT_AREA_NM']).strip() if pd.notna(row['VISIT_AREA_NM']) else ''
        if '집' in name:
            residence_locations.append({
                'VISIT_AREA_ID': row['VISIT_AREA_ID'],
                'VISIT_AREA_NM': name,
                'ROAD_NM_ADDR': row.get('ROAD_NM_ADDR', ''),
                'LOTNO_ADDR': row.get('LOTNO_ADDR', '')
            })
    
    df_residence = pd.DataFrame(residence_locations)
    print(f"'집'이 포함된 장소: {len(df_residence)}개")
    
    if len(df_residence) > 0:
        print("\n거주지 목록 (최대 20개):")
        for i, (_, row) in enumerate(df_residence.iterrows()):
            if i >= 20:
                print(f"... 외 {len(df_residence) - 20}개")
                break
            print(f"- {row['VISIT_AREA_NM']} (ID: {row['VISIT_AREA_ID']})")
    
    return df_residence

def validate_mapping_results(result):
    """매핑 결과 검증 및 디버깅 정보 출력"""
    if not result:
        print("결과가 없습니다.")
        return
    
    print("\n=== 매핑 결과 검증 ===")
    
    # 새 ID 정보 확인
    new_id_info = result.get('new_id_info')
    if new_id_info is not None:
        print(f"\n새로운 ID 정보 (상위 10개):")
        print(new_id_info.head(10))
        
        # 중복이 많은 장소 확인
        print(f"\n같은 이름으로 통합된 장소 (상위 10개):")
        top_duplicates = new_id_info.nlargest(10, 'ORIGINAL_ID_COUNT')
        for _, row in top_duplicates.iterrows():
            print(f"- {row['VISIT_AREA_NM']}: {row['ORIGINAL_ID_COUNT']}개 통합")
    
    # 매핑 상세 정보 확인
    mapping_details = result.get('mapping_details')
    if mapping_details is not None:
        print(f"\n매핑 상세 정보 샘플:")
        print(mapping_details.head(10))
        
        # 거주지 통합 확인
        residence_mapped = mapping_details[mapping_details['IS_RESIDENCE'] == True]
        print(f"\n'집'으로 통합된 ID 수: {len(residence_mapped)}")
        if len(residence_mapped) > 0:
            print("거주지 예시 (최대 10개):")
            for i, (_, row) in enumerate(residence_mapped.head(10).iterrows()):
                print(f"- {row['VISIT_AREA_NM']} (ID: {row['VISIT_AREA_ID']})")
    
    # 관광사진 매핑 결과 확인
    photo_data = result.get('photo_data')
    if photo_data is not None:
        print(f"\n=== 관광사진 매핑 결과 ===")
        print(f"총 사진 수: {len(photo_data)}")
        print(f"매핑 성공: {photo_data['NEW_VISIT_AREA_ID'].notna().sum()}")
        print(f"매핑 실패: {photo_data['NEW_VISIT_AREA_ID'].isna().sum()}")
        
        # 사진이 많은 장소
        photo_by_place = photo_data.groupby(['NEW_VISIT_AREA_ID', 'VISIT_AREA_NM']).size().reset_index(name='photo_count')
        photo_by_place = photo_by_place.sort_values('photo_count', ascending=False)
        print(f"\n사진이 많은 장소 TOP 5:")
        for _, row in photo_by_place.head(5).iterrows():
            place_name = row['VISIT_AREA_NM'] if pd.notna(row['VISIT_AREA_NM']) else '이름없음'
            print(f"- {place_name} (NEW_ID: {int(row['NEW_VISIT_AREA_ID'])}): {row['photo_count']}개")
        
        # '집' 관련 장소 확인
        residence_photos = photo_data[photo_data['VISIT_AREA_NM'].str.contains('집', na=False)]
        residence_photos_filtered = residence_photos[
            ~residence_photos['VISIT_AREA_NM'].str.contains('횟집|술집|찜질|커피|볶는', na=False)
        ]
        
        if len(residence_photos_filtered) > 0:
            print(f"\n=== '집' 관련 장소 분석 ===")
            unique_residence_new_ids = residence_photos_filtered['NEW_VISIT_AREA_ID'].nunique()
            print(f"순수 거주지('집') 사진 수: {len(residence_photos_filtered)}개")
            print(f"고유 NEW_ID 수: {unique_residence_new_ids}개")
            
            if unique_residence_new_ids > 1:
                print("\n❗ 경고: '집'이 여러 NEW_ID로 분산되어 있습니다!")
                residence_distribution = residence_photos_filtered.groupby(['NEW_VISIT_AREA_ID', 'VISIT_AREA_NM']).size().reset_index(name='count')
                residence_distribution = residence_distribution.sort_values('count', ascending=False)
                print("\n거주지 NEW_ID 분포:")
                for _, row in residence_distribution.head(10).iterrows():
                    print(f"  - NEW_ID {int(row['NEW_VISIT_AREA_ID'])}: {row['VISIT_AREA_NM']} ({row['count']}개)")
            else:
                print("✅ 모든 거주지가 하나의 NEW_ID로 통합되었습니다.")

def check_residence_mapping_issue(df_visit, df_photo):
    """거주지 매핑 문제를 진단하는 함수"""
    print("\n=== 거주지 매핑 문제 진단 ===")
    
    # 1. 방문지 데이터에서 '집' 관련 ID 확인
    visit_residence_ids = set()
    for _, row in df_visit.iterrows():
        name = str(row['VISIT_AREA_NM']).strip() if pd.notna(row['VISIT_AREA_NM']) else ''
        if '집' in name and '횟집' not in name and '술집' not in name:
            visit_residence_ids.add(row['VISIT_AREA_ID'])
    
    # 2. 관광사진 데이터에서 '집' 관련 ID 확인
    photo_residence_ids = set()
    for _, row in df_photo.iterrows():
        name = str(row['VISIT_AREA_NM']).strip() if pd.notna(row['VISIT_AREA_NM']) else ''
        if '집' in name and '횟집' not in name and '술집' not in name:
            photo_residence_ids.add(row['VISIT_AREA_ID'])
    
    # 3. 차이 분석
    only_in_photo = photo_residence_ids - visit_residence_ids
    only_in_visit = visit_residence_ids - photo_residence_ids
    in_both = visit_residence_ids & photo_residence_ids
    
    print(f"방문지 데이터의 '집' ID: {len(visit_residence_ids)}개")
    print(f"관광사진 데이터의 '집' ID: {len(photo_residence_ids)}개")
    print(f"양쪽에 모두 있는 ID: {len(in_both)}개")
    print(f"관광사진에만 있는 ID: {len(only_in_photo)}개")
    print(f"방문지에만 있는 ID: {len(only_in_visit)}개")
    
    if only_in_photo:
        print("\n관광사진에만 있는 '집' ID 예시:")
        for photo_id in list(only_in_photo)[:5]:
            photo_info = df_photo[df_photo['VISIT_AREA_ID'] == photo_id].iloc[0]
            print(f"  - ID: {photo_id}, 이름: {photo_info['VISIT_AREA_NM']}")
    
    return only_in_photo

def add_travel_id_to_mapping_details(df_visit, mapping_details):
    """각 매핑 상세 정보에 TRAVEL_ID를 추가하는 함수"""
    visit_id_to_travel_id = df_visit.set_index('VISIT_AREA_ID')['TRAVEL_ID'].to_dict()

    for detail in mapping_details:
        visit_id = detail['VISIT_AREA_ID']
        detail['TRAVEL_ID'] = visit_id_to_travel_id.get(visit_id, '')
    
    return mapping_details

# 실행
if __name__ == "__main__":
    # 데이터 로드
    df_visit = pd.read_csv('./merged_csv/방문지_total.csv')
    df_photo = pd.read_csv('./merged_csv/관광사진_total.csv')
    
    # 거주지 매핑 문제 진단
    check_residence_mapping_issue(df_visit, df_photo)
    
    # '집'이 포함된 장소 확인
    check_residence_places(df_visit)
    
    # 매핑 실행
    result = run_with_default_settings()
    if result:
        validate_mapping_results(result)


=== 거주지 매핑 문제 진단 ===
방문지 데이터의 '집' ID: 1645개
관광사진 데이터의 '집' ID: 642개
양쪽에 모두 있는 ID: 639개
관광사진에만 있는 ID: 3개
방문지에만 있는 ID: 1006개

관광사진에만 있는 '집' ID 예시:
  - ID: 2309190003, 이름: nan
  - ID: 2305160004, 이름: nan
  - ID: 2308120011, 이름: nan

=== '집'이 포함된 장소 확인 ===
'집'이 포함된 장소: 24107개

거주지 목록 (최대 20개):
- 집 (ID: 2304290001)
- 호남 횟집 (ID: 2305010004)
- 집 (ID: 2305020003)
- 집 (ID: 2304280001)
- 집 (ID: 2304280008)
- 본가(부모님 집) (ID: 2304290005)
- 본가(부모님 집) (ID: 2304300005)
- 집 (ID: 2304280001)
- 저녁노을 횟집 (ID: 2304290002)
- 집 (ID: 2304300003)
- 집 (ID: 2305270001)
- 도이동 집 (ID: 2305270003)
- 집 (ID: 2305290004)
- 집 (ID: 2305200001)
- 친척 집 (ID: 2305200002)
- 집 (ID: 2305200006)
- 집 (ID: 2305270001)
- 집 (ID: 2305270006)
- 집 (ID: 2306040003)
- 집 (ID: 2306040005)
... 외 24087개
=== 데이터 로드 시작 ===
방문지 정보: 140555 행, 23 열
이동내역: 140555 행, 11 열
관광사진: 66508 행, 12 열

=== 방문지 정보 샘플 ===
   VISIT_AREA_ID            VISIT_AREA_NM               ROAD_NM_ADDR  \
0     2304290001                        집                 유등로655번길27  

# 매핑 파일 기반으로 매핑 진행

In [30]:
import os
import pandas as pd

# 매핑 결과 파일 로드 (TRAVEL_ID 기반 매핑용!)
mapping_df = pd.read_csv('./merged_csv/fin/id_mapping_details.csv')

# 딕셔너리: (TRAVEL_ID, VISIT_AREA_ID) -> NEW_VISIT_AREA_ID
mapping_df['TRAVEL_ID'] = mapping_df['TRAVEL_ID'].fillna('').astype(str)
mapping_df['VISIT_AREA_ID'] = mapping_df['VISIT_AREA_ID'].astype(str)
pair_to_new_id = mapping_df.set_index(['TRAVEL_ID', 'VISIT_AREA_ID'])['NEW_VISIT_AREA_ID'].to_dict()

# 이름 기반 매핑 딕셔너리 (방문지용)
name_to_new_id = mapping_df.set_index('VISIT_AREA_NM')['NEW_VISIT_AREA_ID'].to_dict()

# 데이터가 있는 최상위 폴더
data_dir = './merged_csv/merged_csv_region/'  # 예시 경로

# 지역 리스트
region_list = [r for r in os.listdir(data_dir) if not r.startswith('.') and os.path.isdir(os.path.join(data_dir, r))]
region_list

['서부권', '동부권', '제주도 및 도서지역', '수도권']

In [36]:
import os
import pandas as pd

# 매핑 결과 파일 로드
mapping_df = pd.read_csv('./merged_csv/fin/id_mapping_details.csv')
mapping_df['TRAVEL_ID'] = mapping_df['TRAVEL_ID'].fillna('').astype(str)
mapping_df['VISIT_AREA_ID'] = mapping_df['VISIT_AREA_ID'].astype(str)
pair_to_new_id = mapping_df.set_index(['TRAVEL_ID', 'VISIT_AREA_ID'])['NEW_VISIT_AREA_ID'].to_dict()
name_to_new_id = mapping_df.set_index('VISIT_AREA_NM')['NEW_VISIT_AREA_ID'].to_dict()

data_dir = './merged_csv/merged_csv_region/'  # 최상위 경로

region_list = [r for r in os.listdir(data_dir) if not r.startswith('.') and os.path.isdir(os.path.join(data_dir, r))]

for region in region_list:
    print(f"\n=== {region} 처리 시작 ===")
    region_path = os.path.join(data_dir, region)
    region_output_dir = os.path.join(region_path, 'fin')  # 변경된 저장 경로!
    os.makedirs(region_output_dir, exist_ok=True)

    # 1️⃣ 방문지 정보 처리
    visit_file = os.path.join(region_path, '방문지_total.csv')
    if os.path.exists(visit_file):
        df_visit = pd.read_csv(visit_file)
        if 'VISIT_AREA_NM' in df_visit.columns:
            df_visit['NEW_VISIT_AREA_ID'] = df_visit['VISIT_AREA_NM'].map(name_to_new_id)
        save_path = os.path.join(region_output_dir, '방문지_fin.csv')
        df_visit.to_csv(save_path, index=False, encoding='utf-8-sig')
        print(f"{region} 방문지_fin.csv 저장 (shape: {df_visit.shape})")

    # 2️⃣ 관광사진 정보 처리
    photo_file = os.path.join(region_path, '관광사진_total.csv')
    if os.path.exists(photo_file):
        df_photo = pd.read_csv(photo_file)
        if 'VISIT_AREA_NM' in df_photo.columns:
            df_photo['NEW_VISIT_AREA_ID'] = df_photo['VISIT_AREA_NM'].map(name_to_new_id)
        save_path = os.path.join(region_output_dir, '관광사진_fin.csv')
        df_photo.to_csv(save_path, index=False, encoding='utf-8-sig')
        print(f"{region} 관광사진_fin.csv 저장 (shape: {df_photo.shape})")

    # 3️⃣ 이동내역 정보 처리
    move_file = os.path.join(region_path, 'fin', '이동내역_fin.csv')
    if os.path.exists(move_file):
        df_move = pd.read_csv(move_file)
        for col in ['START_VISIT_AREA_ID', 'END_VISIT_AREA_ID']:
            if col in df_move.columns:
                df_move[col] = df_move[col].astype(str)
                df_move['TRAVEL_ID'] = df_move['TRAVEL_ID'].fillna('').astype(str)
                df_move[f'NEW_{col}'] = df_move.apply(
                    lambda row: pair_to_new_id.get((row['TRAVEL_ID'], row[col])), axis=1
                )
        save_path = os.path.join(region_output_dir, '이동내역_fin.csv')
        df_move.to_csv(save_path, index=False, encoding='utf-8-sig')
        print(f"{region} 이동내역_fin.csv 저장 (shape: {df_move.shape})")



=== 서부권 처리 시작 ===
서부권 방문지_fin.csv 저장 (shape: (31875, 26))
서부권 관광사진_fin.csv 저장 (shape: (16549, 15))
서부권 이동내역_fin.csv 저장 (shape: (31875, 13))

=== 동부권 처리 시작 ===
동부권 방문지_fin.csv 저장 (shape: (32930, 26))
동부권 관광사진_fin.csv 저장 (shape: (16266, 15))
동부권 이동내역_fin.csv 저장 (shape: (32930, 13))

=== 제주도 및 도서지역 처리 시작 ===
제주도 및 도서지역 방문지_fin.csv 저장 (shape: (51596, 26))
제주도 및 도서지역 관광사진_fin.csv 저장 (shape: (17224, 15))
제주도 및 도서지역 이동내역_fin.csv 저장 (shape: (51596, 13))

=== 수도권 처리 시작 ===
수도권 방문지_fin.csv 저장 (shape: (24154, 26))
수도권 관광사진_fin.csv 저장 (shape: (16469, 15))
수도권 이동내역_fin.csv 저장 (shape: (24154, 13))
